In [9]:
import numpy as np
import os


import tensorflow.config
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from tensorflow.keras.models import load_model


data_left = np.load("F:\project1\subj01\\training_split\\training_fmri\lh_training_fmri.npy")
data_right = np.load("F:\project1\subj01\\training_split\\training_fmri\\rh_training_fmri.npy")
# np.shape(data_left) == (9841, 19004)
# np.shape(data_right) == (9841, 20544)


# Loads all the fmri data associated to a given subject
def load_fmri_of_subject(hemisphere='left'):
  if hemisphere == 'left':
    brain_activity = np.load("F:\project1\subj01\\training_split\\training_fmri\lh_training_fmri.npy")
  else:
    brain_activity = np.load("F:\project1\subj01\\training_split\\training_fmri\\rh_training_fmri.npy")
  return brain_activity


print(data_left[0])


# Lấy caption
caption_folder_path =  'F:\project1\subj01\\training_split\\training_caption'
all_captions = []
# Lặp qua từng file trong thư mục
for file_name in os.listdir(caption_folder_path):
    file_path = os.path.join(caption_folder_path, file_name)
    
    # Kiểm tra nếu là file .txt
    if file_name.endswith('.txt'):
        with open(file_path, 'r') as file:
            caption = file.read().strip()  # Đọc nội dung và loại bỏ khoảng trắng ở hai đầu
            all_captions.append(caption)  # Thêm nội dung vào danh sách all_captions
print(all_captions)
# len(all_captions) == 9841


# tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)
word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
max_length = max(len(caption.split()) for caption in all_captions)

#max_length == 75
#vocab_size == 1780

[-0.8617882  -0.20318632 -0.62639767 ... -1.1170396  -0.965407
 -0.09847511]
['startseq man in blue shirt is sitting at desk with his head propped up in front of him endseq', 'startseq man in blue shirt is sitting on the floor in front of large display of bread endseq', 'startseq woman in white apron and white apron is sitting at table with various ingredients on her side endseq', 'startseq man in wetsuit is riding surfboard endseq', 'startseq man is standing in the air in front of large metal structure endseq', 'startseq woman in pink shirt is drumming with flour on the end of the street endseq', 'startseq woman is washing tub of water endseq', 'startseq man is sleeping on bench in front of large building endseq', 'startseq bird flies through the air endseq', 'startseq man in blue shirt is riding bicycle down the street endseq', 'startseq two people are walking along the beach endseq', 'startseq woman in pink shirt is sitting on the floor endseq', 'startseq bird swoops down hall endse

In [66]:
import numpy as np
import pickle
from keras.models import Sequential
from keras.layers import Dense

# Define the model architecture
brain_extract_model = Sequential()
brain_extract_model.add(Dense(4096, activation='relu', input_shape=(19004,)))

# Compile the model
brain_extract_model.compile(optimizer='adam', loss='mse')

# Print model summary
print(brain_extract_model.summary())

# Function to extract features and save to a .pkl file
def extract_features_and_save(input_data, output_file):
    # Extract features
    features = brain_extract_model.predict(input_data)
    # Save features to a .pkl file
    with open(output_file, 'wb') as f:
        pickle.dump(features, f)

# Example usage
# Replace 'input_data' with your actual input data array
# Replace 'output.pkl' with the desired output .pkl file path
#output_file = 'brain.pkl'  # Example output file path
#extract_features_and_save(data_left, output_file)


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 4096)              77844480  
                                                                 
Total params: 77844480 (296.95 MB)
Trainable params: 77844480 (296.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [22]:
# load features
with open('brain.pkl', 'rb') as f:
    features = pickle.load(f)

In [23]:
np.shape(features)

(9841, 4096)

In [24]:
# create data generator to get data in batch (avoids session crash)
def data_generator(data_keys , tokenizer, features, max_length, vocab_size, batch_size):
    X1, X2, y = list(), list(), list()
    n = 0
    while 1:
        for key in range(len(data_keys)):
            n += 1
            caption = all_captions[key]
            # encode the sequence
            seq = tokenizer.texts_to_sequences([caption])[0]
            # split the sequence into X, y pairs
            for i in range(1, len(seq)):
                # split into input and output pairs
                in_seq, out_seq = seq[:i], seq[i]
                # pad input sequence
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                # encode output sequence
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                
                # store the sequences
                X1.append(features[key])
                X2.append(in_seq)
                y.append(out_seq)
            if n == batch_size:
                #X1, X2, y = np.array(X1), np.array(X2), np.array(y)
                #yield {"brain_activity": X1, "text": X2}, y
                yield [np.array(X1), np.array(X2)], np.array(y)
                X1, X2, y = list(), list(), list()
                n = 0

In [25]:
### encoder model
## brain activity feature layers
# brain activity input
inputs1 = Input(shape=(4096,), name="brain_activity")  # 4096 features
fe1 = Dropout(0.4)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

## text feature layers
inputs2 = Input(shape=(max_length,), name="text")          # max_length = 75 - max length of a comment in dataset
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)  # vocab_size == 1780
se2 = Dropout(0.4)(se1)
se3 = LSTM(256)(se2)

### decoder model
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [26]:
split = int(len(all_captions) * 0.95)
split = 9500

train_caption = all_captions[:split] # == 8856 array
test_caption = all_captions[split:] # == 985

brain_activity = load_fmri_of_subject('left')
train_brain_activity = brain_activity[:split] # == (8856,19004)
test_brain_activity = brain_activity[split:]

In [28]:
# Save model
#model.save('brain_lstm_model2.h5')
# Load model
#model = load_model("brain_lstm_model2.h5")

f:\project1\env\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [97]:
def idx_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

# generate caption for an image
def predict_caption(model, brain_feature, tokenizer, max_length):
    # add start tag for generation process
    in_text = 'startseq'
    # iterate over the max length of sequence
    for i in range(max_length):
        # encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        # pad the sequence
        sequence = pad_sequences([sequence], max_length)
        # predict next word
        yhat = model.predict([brain_feature, sequence], verbose=0)
        # get index with high probability
        yhat = np.argmax(yhat)
        # convert index to word
        word = idx_to_word(yhat, tokenizer)
        # stop if word not found
        if word is None:
            break
        # append word as input for generating next word
        in_text += " " + word
        # stop if we reach end tag
        if word == 'endseq':
            break
      
    return in_text

# generate caption from a brain activity
def generate_caption(test_brain_activity_instance):
    test_feature = brain_extract_model.predict(np.expand_dims(test_brain_activity_instance, axis=0))
    return predict_caption(model, test_feature, tokenizer, max_length)


In [114]:
# test data is from test_brain_activity[9500:]
test_brain_activity_instance = test_brain_activity[-7]
print(generate_caption(test_brain_activity_instance))

1/1 [==============================] - 0s 21ms/step
startseq man in blue shirt is sitting on the ground in front of an old fashioned truck endseq


In [115]:
import os
import glob

# Đường dẫn tới thư mục chứa các file .txt
folder_path = "F:\project1\subj01\\training_split\\training_caption"

# Tạo một list để lưu trữ nội dung của các file .txt
text_data = []

# Lặp qua tất cả các file trong thư mục
for file_path in glob.glob(os.path.join(folder_path, "*.txt")):
    # Đọc nội dung của file và thêm vào list
    with open(file_path, "r", encoding="utf-8") as file:
        text = file.readline().strip()  # Đọc chỉ 1 dòng text trong file
        text_data.append(text)

# In ra số lượng dòng text đã đọc và kiểm tra dữ liệu
print("Số lượng dòng text:", len(text_data))
print("Các dòng text đã đọc:", text_data)


Số lượng dòng text: 9841
Các dòng text đã đọc: ['startseq man in blue shirt is sitting at desk with his head propped up in front of him endseq', 'startseq man in blue shirt is sitting on the floor in front of large display of bread endseq', 'startseq woman in white apron and white apron is sitting at table with various ingredients on her side endseq', 'startseq man in wetsuit is riding surfboard endseq', 'startseq man is standing in the air in front of large metal structure endseq', 'startseq woman in pink shirt is drumming with flour on the end of the street endseq', 'startseq woman is washing tub of water endseq', 'startseq man is sleeping on bench in front of large building endseq', 'startseq bird flies through the air endseq', 'startseq man in blue shirt is riding bicycle down the street endseq', 'startseq two people are walking along the beach endseq', 'startseq woman in pink shirt is sitting on the floor endseq', 'startseq bird swoops down hall endseq', 'startseq man in blue shir

In [117]:
text_data[20:50]

['startseq two people are jumping in the air in front of large building endseq',
 'startseq black dog is playing with ball in the grass endseq',
 'startseq group of people are hiking through valley endseq',
 'startseq man in glasses is sitting at table with laptop in front of him endseq',
 'startseq man in bathrobe is sitting at desk with his hands in his hands endseq',
 'startseq young girl is sitting in highchair with spoon in her mouth endseq',
 'startseq man in black shirt is standing on the sidewalk in front of an old building endseq',
 'startseq man in wetsuit is riding his surfboard in the ocean endseq',
 'startseq man in blue shirt is sleeping on bus endseq',
 'startseq people are standing on street corner with buildings in the background endseq',
 'startseq solitary passenger is arriving at the airport endseq',
 'startseq two people are riding on the grass in the desert endseq',
 'startseq an asian woman is mixing ingredients in bowl endseq',
 'startseq man in black shirt is s